# Что будет
- сводные таблицы
- фильтры и вычисления с помощью метода loc
- фильтрация пустых значений через isnull
- время в pandas
- строковые методы
- немного про учет форм слов

# Сводные таблицы
Прям как в экселе

In [3]:
import pandas as pd

In [4]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


### Упражнение
Какие оценки поставил пользователь №1 и в каком количестве?

Что делаем:
1. Фильтруем датафрейм ratings для userId = 1
2. Считаем для этого пользователя сколько он выставил единиц, двоек итд

In [5]:
ratings[ratings.userId == 1].rating.value_counts()

2.0    7
3.0    4
4.0    3
2.5    3
1.0    2
3.5    1
Name: rating, dtype: int64

In [6]:
filtred_ratings = ratings[ (ratings['userId'] == 1) ]
filtred_ratings['rating'].value_counts()

2.0    7
3.0    4
4.0    3
2.5    3
1.0    2
3.5    1
Name: rating, dtype: int64

In [13]:
ratings[ratings['userId'] == 1].groupby('userId').count()

,movieId,rating,timestamp
userId,,,
1,20,20,20


Как это сделать для всех пользователей сразу:

In [7]:
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0).head()

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
userId,,,,,,,,,,
1,0,2,0,7,3,4,1,3,0,0
2,0,2,0,4,0,36,0,23,0,11
3,0,0,0,1,3,18,9,11,4,5
4,0,5,0,5,0,23,0,52,0,119
5,0,0,1,0,3,3,27,42,19,5


In [8]:
# можно итоги добавить
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0, 
                    margins = True).head()

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,All
userId,,,,,,,,,,,
1,0,2,0,7,3,4,1,3,0,0,20
2,0,2,0,4,0,36,0,23,0,11,76
3,0,0,0,1,3,18,9,11,4,5,51
4,0,5,0,5,0,23,0,52,0,119,204
5,0,0,1,0,3,3,27,42,19,5,100


In [18]:
# можно итоги добавить
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0, 
                    margins = True).tail().reset_index()

rating,userId,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,All
0,668,0,2,0,0,0,7,0,3,0,8,20
1,669,0,0,0,7,0,14,0,12,0,4,37
2,670,0,2,0,3,0,5,0,10,0,11,31
3,671,0,1,0,4,4,9,13,57,8,19,115
4,All,1101,3326,1687,7271,4449,20064,10538,28750,7723,15095,100004


In [17]:
type(ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0, 
                    margins = True).tail())

pandas.core.frame.DataFrame

### Упражнение
Какой пользователь выставил больше всех пятерок?

In [8]:
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0)
.sort_values(5, ascending=False).head()

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
userId,,,,,,,,,,
564,0,152,0,187,0,414,0,707,0,408
232,0,35,0,32,0,96,0,276,0,243
242,0,0,0,4,0,25,0,151,0,219
547,53,79,58,204,177,411,378,591,226,214
30,0,20,0,86,0,201,2,505,1,196


### Фильтры и вычисления с помощью loc и iloc

In [9]:
log = pd.read_csv('visit_log.csv', sep=';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


### Но сначала упражнение
Сколько различных страниц есть в столбце url? Постройте топ-5 страниц по посещаемости.

Бонусный вариант: выведите топ-5 страниц без указания хоста https://host.ru

In [10]:
log['url_new'] = log.url.str.replace('https://host.ru/', '')
log['url_new'].value_counts().head()

3c19b4ef7371864fa3    541
108ce4b365afb7b88e    534
23c64743ad842b9311    532
1afc5618562365020a    529
2fcb223266880ccf57    526
Name: url_new, dtype: int64

In [11]:
log.url.apply(lambda x: x[16:])

0        3c19b4ef7371864fa3
1        c8d9213a31839f9a3a
2        b8b58337d272ee7b15
3        b8b58337d272ee7b15
4        b8b58337d272ee7b15
                ...        
18933    c2382eb3d6afc8d0f3
18934    f1eb4601740d627ab0
18935    a5dda93e70318570c0
18936    6fda01ec57f23abc9e
18937    39fa884393666d45fc
Name: url, Length: 18938, dtype: object

In [37]:
# метод loc позволяет выбрать строки и столбцы в соответствии с условиями
# двоеточие означает выбор всех значений

log.loc[:, ['user_id', 'region']].head()

,user_id,region
0,b1613cc09f,Russia
1,4c3ec14bee,Russia
2,a8c40697fb,Russia
3,521ac1d6a0,Russia
4,d7323c571c,Russia


In [12]:
# iloc работает аналогично, но с указанием номера строки / столбца

log.iloc[:, [1, -3]].head()

,visit_id,user_id
0,e3b0c44298,b1613cc09f
1,6e340b9cff,4c3ec14bee
2,96a296d224,a8c40697fb
3,709e80c884,521ac1d6a0
4,df3f619804,d7323c571c


In [13]:
# пример фильтра на страну
# если столбцы не надо фильтровать, то второй параметр можно не указывать

log.loc[log.region == 'Russia'].head()

,timestamp,visit_id,url,region,user_id,traffic_source,url_new
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,3c19b4ef7371864fa3
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,c8d9213a31839f9a3a
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,b8b58337d272ee7b15
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,b8b58337d272ee7b15
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,b8b58337d272ee7b15


In [14]:
# пример вычисления нового столбца с НДС для страны

log.loc[log.region == 'Russia', 'VAT'] = 1.2
log.head(10)

,timestamp,visit_id,url,region,user_id,traffic_source,url_new,VAT
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,3c19b4ef7371864fa3,1.2
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,c8d9213a31839f9a3a,1.2
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,b8b58337d272ee7b15,1.2
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,b8b58337d272ee7b15,1.2
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,b8b58337d272ee7b15,1.2
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,df646c3676cc259fa0,1.2
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,b8b58337d272ee7b15,1.2
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,108ce4b365afb7b88e,1.2
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,3004a8273caeef2867,NaN
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,df646c3676cc259fa0,1.2


In [41]:
# вариант с вычисляемым фильтром

log.loc[lambda row: row.region == 'Russia'].head(10)

,timestamp,visit_id,url,region,user_id,traffic_source,url_new,VAT
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,3c19b4ef7371864fa3,1.2
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,c8d9213a31839f9a3a,1.2
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,b8b58337d272ee7b15,1.2
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,b8b58337d272ee7b15,1.2
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,b8b58337d272ee7b15,1.2
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,df646c3676cc259fa0,1.2
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,b8b58337d272ee7b15,1.2
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,108ce4b365afb7b88e,1.2
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,df646c3676cc259fa0,1.2
12,1549980789,15ec7bf0b5,https://host.ru/8e88d4703848cc0ec4,Russia,6d1d81f7f8,paid,8e88d4703848cc0ec4,1.2


### Скорость метода loc
Посчитаем средний рейтинг разных жанров. Метод merge взят из материалов следующего занятия ¯ \ _ (ツ) _ / ¯

In [15]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
joined = ratings.merge(movies, how='left', on='movieId')
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [16]:
joined['Adventure'] = joined.apply(lambda row: row.rating if 'Adventure' in row.genres else None, axis=1)
joined.head()

,userId,movieId,rating,timestamp,title,genres,Adventure
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,NaN
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,NaN
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,NaN
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,2.0
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,NaN


Можно и через loc

In [44]:
joined['Adventure'] = joined.loc[joined.genres.str.contains('Adventure'), 'rating']
joined.head()

,userId,movieId,rating,timestamp,title,genres,Adventure
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,NaN
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,NaN
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,NaN
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,2.0
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,NaN


Для замера времени сделаем эти операции для набор жанров

In [17]:
genres = ['Adventure', 'Animation', 'Children', 'Drama', 'Musical', 'Thriller']

In [21]:
%%time
joined = ratings.merge(movies, how='left', on='movieId')

for genre in genres:
    joined[genre] = joined.apply(lambda row: row.rating if genre in row.genres else None, axis=1)
joined.head()

Wall time: 25.2 s


,userId,movieId,rating,timestamp,title,genres,Adventure,Animation,Children,Drama,Musical,Thriller
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,NaN,NaN,NaN,2.5,NaN,NaN
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,NaN,3.0,3.0,3.0,3.0,NaN
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,NaN,NaN,NaN,NaN,NaN,3.0
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,2.0,NaN,NaN,NaN,NaN,2.0
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,NaN,NaN,NaN,4.0,NaN,NaN


In [22]:
%%time
joined = ratings.merge(movies, how='left', on='movieId')

for genre in genres:
    joined[genre] = joined.loc[joined.genres.str.contains(genre), 'rating']
joined

Wall time: 642 ms


,userId,movieId,rating,timestamp,title,genres,Adventure,Animation,Children,Drama,Musical,Thriller
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,NaN,NaN,NaN,2.5,NaN,NaN
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,NaN,3.0,3.0,3.0,3.0,NaN
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,NaN,NaN,NaN,NaN,NaN,3.0
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,2.0,NaN,NaN,NaN,NaN,2.0
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,NaN,NaN,NaN,4.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
99999,671,6268,2.5,1065579370,Raising Victor Vargas (2002),Comedy|Drama|Romance,NaN,NaN,NaN,2.5,NaN,NaN
100000,671,6269,4.0,1065149201,Stevie (2002),Documentary,NaN,NaN,NaN,NaN,NaN,NaN
100001,671,6365,4.0,1070940363,"Matrix Reloaded, The (2003)",Action|Adventure|Sci-Fi|Thriller|IMAX,4.0,NaN,NaN,NaN,NaN,4.0
100002,671,6385,2.5,1070979663,Whale Rider (2002),Drama,NaN,NaN,NaN,2.5,NaN,NaN


### Упражнение
Какие варианты источников трафика есть в столбце traffic_source?

Создайте столбец traffic_type, в котором для источников 'yandex' и 'google' будет стоять значение 'organic'. А для остальных вариантов - NaN.

In [49]:
log.loc[(log.traffic_source == 'yandex') | (log.traffic_source == 'google'), 'traffic_type'] = 'organic'
log.head(10)

,timestamp,visit_id,url,region,user_id,traffic_source,url_new,VAT,traffic_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,3c19b4ef7371864fa3,1.2,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,c8d9213a31839f9a3a,1.2,NaN
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,b8b58337d272ee7b15,1.2,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,b8b58337d272ee7b15,1.2,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,b8b58337d272ee7b15,1.2,organic
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,df646c3676cc259fa0,1.2,organic
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,b8b58337d272ee7b15,1.2,NaN
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,108ce4b365afb7b88e,1.2,NaN
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,3004a8273caeef2867,NaN,NaN
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,df646c3676cc259fa0,1.2,NaN


In [51]:
log.loc[log.traffic_source.isin(['yandex', 'google']), 'traffic_type'] = 'organic'
log.head(10)

,timestamp,visit_id,url,region,user_id,traffic_source,url_new,VAT,traffic_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,3c19b4ef7371864fa3,1.2,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,c8d9213a31839f9a3a,1.2,NaN
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,b8b58337d272ee7b15,1.2,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,b8b58337d272ee7b15,1.2,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,b8b58337d272ee7b15,1.2,organic
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,df646c3676cc259fa0,1.2,organic
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,b8b58337d272ee7b15,1.2,NaN
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,108ce4b365afb7b88e,1.2,NaN
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,3004a8273caeef2867,NaN,NaN
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,df646c3676cc259fa0,1.2,NaN


### Методы isnull, isna
Определение пустых или None значений. По сути одинаковые методы

In [52]:
import numpy as np

In [53]:
df = pd.DataFrame({'value': [123, None, np.nan, np.NaN, np.NAN, 456]})
df

,value
0,123.0
1,NaN
2,NaN
3,NaN
4,NaN
5,456.0


In [56]:
# фильтр на пустые значения в столбце value

df.loc[pd.isnull(df.value), :]

,value
1,NaN
2,NaN
3,NaN
4,NaN


### Упражнение
Для пустых значений в столбце traffic_type выставьте значение 'other'

### Дата и время в pandas

In [23]:
log['date'] = pd.to_datetime(log['timestamp'], unit='s')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,url_new,VAT,date
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,3c19b4ef7371864fa3,1.2,2019-02-12 14:11:32
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,c8d9213a31839f9a3a,1.2,2019-02-12 14:11:44
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,b8b58337d272ee7b15,1.2,2019-02-12 14:11:55
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,b8b58337d272ee7b15,1.2,2019-02-12 14:12:05
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,b8b58337d272ee7b15,1.2,2019-02-12 14:12:16


In [24]:
log['date'] = pd.to_datetime(log['timestamp'], format='%Y-%m-%dT%H')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,url_new,VAT,date
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,3c19b4ef7371864fa3,1.2,1970-01-01 00:00:01.549980692
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,c8d9213a31839f9a3a,1.2,1970-01-01 00:00:01.549980704
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,b8b58337d272ee7b15,1.2,1970-01-01 00:00:01.549980715
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,b8b58337d272ee7b15,1.2,1970-01-01 00:00:01.549980725
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,b8b58337d272ee7b15,1.2,1970-01-01 00:00:01.549980736


In [25]:
# столбец datetime64[ns] теперь имеет тип даты
log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18938 entries, 0 to 18937
Data columns (total 9 columns):
timestamp         18938 non-null int64
visit_id          18938 non-null object
url               18938 non-null object
region            18938 non-null object
user_id           18938 non-null object
traffic_source    18938 non-null object
url_new           18938 non-null object
VAT               10913 non-null float64
date              18938 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(6)
memory usage: 1.3+ MB


In [26]:
# получим час визита

log['hour'] = log.date.dt.hour
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,url_new,VAT,date,hour
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,3c19b4ef7371864fa3,1.2,1970-01-01 00:00:01.549980692,0
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,c8d9213a31839f9a3a,1.2,1970-01-01 00:00:01.549980704,0
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,b8b58337d272ee7b15,1.2,1970-01-01 00:00:01.549980715,0
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,b8b58337d272ee7b15,1.2,1970-01-01 00:00:01.549980725,0
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,b8b58337d272ee7b15,1.2,1970-01-01 00:00:01.549980736,0


### Упражнение
Дан датафрейм df с временем визита и соответствующей покупки. Сделайте следующее:
1. Создайте столбцы visit_dt и purchase_dt типа datetime из столбцов visit_datetime и purchase_datetime
2. Посчитайте разницу между временем визита и покупки (простым вычитанием)
3. Переведите полученный столбец с длительностью покупки в минуты:
```python
(df.purchase_dt - df.visit_dt).astype('timedelta64[m]')
```
4. Посчитайте среднее время покупки в минутах

In [ ]:
df = pd.DataFrame({'visit_datetime': ['2019-11-04T00:05:13', '2019-11-04T00:06:19', '2019-11-04T01:35:14'], 
                   'purchase_datetime': ['2019-11-04T00:15:53', '2019-11-04T00:07:59', '2019-11-04T03:15:30']})
df

### Методы работы со строками

In [28]:
stats = pd.read_csv('keywords.csv')
stats.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


Проверка наличия подстроки в строке в питоне:

In [29]:
'охотник' in 'каждый охотник желает знать...'

True

Аналог в pandas:

In [30]:
stats[stats.keyword.str.contains('охотник')].head()

,keyword,shows
3072,сумеречные охотники 2 сезон,71965
3474,сумеречные охотники,66083
3654,белоснежка и охотник 2 фильм 2016,63154
4178,последний охотник на ведьм,57560
6127,последний охотник на ведьм фильм 2015,42213


[Документация](https://www.geeksforgeeks.org/python-pandas-series-str-contains/)

Syntax: Series.str.contains(pat, case=True, flags=0, na=nan, regex=True)

Parameter :
- pat : Character sequence or regular expression.
- case : If True, case sensitive.
- flags : Flags to pass through to the re module, e.g. re.IGNORECASE.
- na : Fill value for missing values.
- regex : If True, assumes the pat is a regular expression.

In [31]:
# поиск одного из нескольких слов

stats[stats.keyword.str.contains('охотник|фильм|2016')].head()

,keyword,shows
20,фильмы 2016,4486635
51,фильмы,2156641
54,фильмы онлайн,2305540
68,смотреть фильмы онлайн,1928484
86,порно фильмы,1458031


### Упражнение
Отфильтруйте датафрейм stats по поисковым запросам, которые содержат строку "погода в" и упоминают один из городов: Москва, Новосибирск, Краснодар.

### replace

In [63]:
'отпуск начнется завтра'.replace('завтра', 'через месяц')

'отпуск начнется через месяц'

Аналог в pandas на запросах про сериалы:

In [32]:
serial = stats[stats.keyword.str.contains('сериалы')]
serial.head()

,keyword,shows
246,сериалы,587779
304,сериалы тут,503967
555,турецкие сериалы на русском языке,296403
881,русские сериалы,234262
890,сериалы онлайн,204812


In [65]:
serial.keyword.str.replace('сериалы', 'книги').head()

246                              книги
304                          книги тут
555    турецкие книги на русском языке
881                      русские книги
890                       книги онлайн
Name: keyword, dtype: object

### Как учитывать разное написание слов
Самое простое - методы upper и lower

In [33]:
serial.keyword.str.upper().head()

246                              СЕРИАЛЫ
304                          СЕРИАЛЫ ТУТ
555    ТУРЕЦКИЕ СЕРИАЛЫ НА РУССКОМ ЯЗЫКЕ
881                      РУССКИЕ СЕРИАЛЫ
890                       СЕРИАЛЫ ОНЛАЙН
Name: keyword, dtype: object

In [34]:
serial.keyword.str.lower().head()

246                              сериалы
304                          сериалы тут
555    турецкие сериалы на русском языке
881                      русские сериалы
890                       сериалы онлайн
Name: keyword, dtype: object

### Что делать если нужно учесть формы написания слов?

In [68]:
stats[stats.keyword.str.contains('рубл')].head()

,keyword,shows
1202,алиэкспресс на русском в рублях официальный сайт,161553
1602,курс гривны к рублю,125076
1736,али экспресс русская версия на русском в рубля...,117260
2132,доллары в рубли,114173
2172,100 долларов в рублях,97534


Почему нельзя просто оставить str.contains('рубл'):

In [ ]:
non_financial_search = 'рубленая котлетка'

### Библиотека [pymystem](https://pypi.org/project/pymystem3/)

In [1]:
from pymystem3 import Mystem

In [2]:
search = 'курс гривны к рублю рубли рублях'

In [3]:
m = Mystem()
lemmas = m.lemmatize(search)
lemmas

Installing mystem to C:\Users\computer/.local/bin\mystem.exe from http://download.cdn.yandex.net/mystem/mystem-3.1-win-64bit.zip


['курс',
 ' ',
 'гривна',
 ' ',
 'к',
 ' ',
 'рубль',
 ' ',
 'рубль',
 ' ',
 'рубль',
 '\n']

In [4]:
' '.join(lemmas)

'курс   гривна   к   рубль   рубль   рубль \n'

### Домашнее задание 3
В датафрейме data создайте столбец lemmas, в котором вычислите леммы поисковых запросов из столбца keyword. Леммы должны иметь строковый тип.

In [5]:
data = pd.DataFrame({
    'keyword': ['курс гривны к рублю', 'доллары в рубли', '100 долларов в рублях', 'курс рубля'],
    'shows': [125076, 114173, 97534, 53546],
})

NameError: name 'pd' is not defined